In [43]:
import torch
import torchvision.models as models

In [49]:
def fetch_model(name):
    #model = getattr(models, name)(pretrained=True)
    if name == "resnet18":
        default_weights = ResNet18_Weights.DEFAULT
        model = models.resnet18(weights=default_weights)    
        model.eval()
        return model, default_weights

model, weights = fetch_model("resnet18")

In [50]:
def save_to_jit(model, name="resnet18_torchscript.pt"):
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save(name) # Save

    
def save_to_pt(model, name="resnet18.pt"):
    torch.save(model.state_dict(), name)

    
def save_to_onnx(model, weights, name="resnet18.onnx"):

    img = read_image("/Users/zjy/Downloads/511px-Grace_Hopper.jpg")

    model.eval()

    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()

    # Step 3: Apply inference preprocessing transforms
    batch = preprocess(img).unsqueeze(0)

    torch.onnx.export(model, batch, name, verbose=True)

save_to_onnx(model, weights, name="production_resnet18.onnx")
#save_to_jit(model)

Exported graph: graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(1000, 512, strides=[512, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_193 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %onnx::Conv_194 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_196 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_197 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_199 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_200 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_202 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_203 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_205 : Fl

In [52]:
img = read_image("/Users/zjy/Downloads/511px-Grace_Hopper.jpg")

# Step 1: Initialize model with the best available weights
#weights = ResNet18_Weights.DEFAULT
#model = resnet18(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

military uniform: 57.7%
